In [1]:
from unsloth import FastLanguageModel
import torch

MODEL = 'unsloth/Qwen3-14B'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    full_finetuning=False,
)

model = FastLanguageModel.get_peft_model(
    model,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    use_gradient_checkpointing='unsloth',
    r=16,
    lora_alpha=4,
    lora_dropout=0,
    bias='none'
)

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

Load reasoning data

In [ ]:
from datasets import load_dataset

name = 'unsloth/OpenMathReasoning-mini'
reasoning_data = load_dataset(name, split='cot')

Load non-reasoning data

In [ ]:
from datasets import load_dataset

name = 'mlabonne/FineTome-100k'
non_reasoning_data = load_dataset(name, split='train')

Prepare reasoning data

In [ ]:
def generate_conv(examples):
    problems = examples['problem']
    solutions = examples['generated_solution']
    conversations = []

    for prob, sol in zip(problems, solutions):
        conversations.append([
            {'role': 'user', 'content': prob},
            {'role': 'assistant', 'content': sol},
        ])

    return conversations


reasoning_conv = generate_conv(reasoning_data)

Prepare non-reasoning data

In [ ]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(non_reasoning_data)
non_reasoning_conv = tokenizer.apply_chat_template(dataset['conversation'])

Define trainer

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    # ...
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=60,
        learning_rate=2e-4,
        # ...
        optim='adamw_8bit',
        weight_decay=0.01,
    )
)

Train

In [ ]:
trainer_stats = trainer.train()